<a href="https://colab.research.google.com/github/lorenzopalaia/Progetto-Lab-IA/blob/main/Progetto_Lab_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Audio Style Transfer & Genre Classification

Questo progetto si propone come obiettivo quello di *applicare uno stile musicale* di una fonte audio, che chiameremo **target**, ad una seconda fonte audio, che chiameremo **source**. In aggiunta intendiamo *classificare i generi musicali* di entrambe le fonti audio.

## Audio Style Transfer

Per raggiungere il primo obiettivo ci avvaliamo della tecnica di Neural Style Transfer come segue:
1. Trasformazione delle fonti audio in spettrogrammi
2. Applicazione del Neural Style Transfer sullo spettrogramma **source** basandoci sullo spettrogramma **target**
3. Trasformazione dello spettrogramma risultante in fonte audio

### Spettrogramma

Uno spettrogramma è la rappresentazione grafica dell'intensità di un suono in funzione del tempo e della frequenza.
* sull'asse delle ascisse è riportato il tempo in scala lineare
* sull'asse delle ordinate è riportata la frequenza in scala lineare o logaritmica
* a ciascun punto di data ascissa e data ordinata è assegnata una tonalità di grigio, o un colore, rappresentante l'intensità del suono in un dato istante di tempo e a una data frequenza

![](https://upload.wikimedia.org/wikipedia/commons/7/70/SpettrogrammaParolaManoColore.jpg)

## Genre Classification

# Dataset

Per conseguire i nostri obiettivi ci avvaliamo del dataset GTZAN scaricabile [qui](https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification). Tutte le fonti audio hanno le seguenti caratteristiche:
* Canale `mono`
* Campionamento a `22 KHz`
* `16 bit per campione`
* Durata `30 s`

Il dataset è strutturato come segue:
* in `genres_original` abbiamo 10 sottocartelle contenenti 100 fonti audio in formato `.wav` per ciascun genere
* in `images_original` abbiamo 10 sottocartelle contenenti i rispettivi spettrogrammi
* 2 file `.csv` contenenti le features delle fonti audio. Un file contiene per ogni brano (di 30 secondi) una media e una varianza calcolata su più features che possono essere estratte da un file audio. L'altro file ha la stessa struttura, ma prima le canzoni sono state suddivise in file audio di 3 secondi (aumentando in questo modo di 10 volte la quantità di dati che potremo fornire al nostro modello di classificazione)

# Setup

Spostiamoci nella cartella di Google Drive dove si trova il nostro dataset

In [12]:
%cd /content/drive/My Drive/Colab Notebooks/GTZAN Dataset

/content/drive/My Drive/Colab Notebooks/GTZAN Dataset


Importiamo i pacchetti che ci servono

In [16]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import numpy as np
from IPython.display import display, Audio
import torch.nn as nn
import torch
from torch.autograd import Variable
import librosa

style_audio_name = 'genres_original/blues/blues.00000.wav'
content_audio_name = 'genres_original/hiphop/hiphop.00000.wav'

# Utilities

Cominciamo a definire una serie di funzioni:
* `toSpectrogram` per ottenere lo spettrogramma di una fonte audio
* `plotSpectrogram` per stampare lo spettrogramma
* `play` per ascoltare la fonte audio

In [14]:
def to_spectrogram(filepath):
  sample_rate, samples = wavfile.read(filepath)
  frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)
  return frequencies, times, spectrogram

In [15]:
def plot_spectrogram(frequencies, times, spectrogram):
  plt.pcolormesh(times, frequencies, 10*np.log10(spectrogram))
  plt.ylabel('Frequency [Hz]')
  plt.xlabel('Time [sec]')
  plt.show()

In [10]:
def play(filepath):
  audio = Audio(filepath)
  display(audio)

Quindi testiamone il funzionamento

In [ ]:
filepath = style_audio_name
frequencies, times, spectrogram = to_spectrogram(filepath)
plot_spectrogram(frequencies, times, spectrogram)
play(filepath)

# Funzioni di Loss

Per questo approccio individuiamo due funzioni di Loss:
1. Content Loss: minimizzarne il valore significa che la fonte audio in uscita suonerà in maniera simile alla fonte di contenuto
2. Stlye Loss: minimizzarne il valore significa che la fonte audio in uscita suonerà in maniera simile alla fonte di stile

Idealmente vorremmo che entrambe fossero minimizzate

## Content Loss

La funzione di Content Loss prende una matrice di input ed un matrice di contenuto che corrisponde alla fonte audio di contenuto. Quindi ritorna la distanza pesata $w_{CL} \cdot D^L_C(X,C)$ tra la matrice di input $X$ e la matrice di contenuto $C$. Implementiamo il tutto estendendo la classe `nn.Module` e sfruttando la funzione `nn.MSELoss`

In [4]:
class ContentLoss(nn.Module):

  def __init__(self, target, weight):
    # costruttore della classe nn.Module da cui deriviamo
    super(ContentLoss, self).__init__()
    # facciamo una detach, necessaria per calolare dinamicamente il gradiente
    self.target = target.detach() * weight
    self.weight = weight
    self.criterion = nn.MSELoss()

  def forward(self, input):
    self.loss = self.criterion(input * self.weight, self.target)
    self.output = input
    return self.output

  def backward(self, retain_graph=True):
    self.loss.backward(retain_graph=retain_graph)
    return self.loss

## Style Loss

Ovviamente vogliamo estrarre dalla fonte di stile solamente le features più importanti. Se ad esempio è presente una parte cantata a noi non interessa. Al contrario vogliamo estrarre solamente la parte 'melodica' con le sue proprietà quali timbro e tono. Dobbiamo allora ricorrere ad una Gram Matrix. Allora prendiamo una prima parte della matrice di input ed eseguiamo una `flatten` per rimuovere una buona parte delle informazioni audio. Ripetiamo lo stesso per un'altra parte della matrice di input. Eseguiamo quindi il prodotto scalare tra le matrici 'appiattite'

![](https://www.w3resource.com/w3r_images/numpy-manipulation-ndarray-flatten-function-image-1.png)

Ma perché scegliamo proprio il prodotto scalare? Perché fornisce una misura di quanto due matrici siano simili o meno. Infatti se le matrici sono fortemente simili tra di loro otterremo un risultato molto grande, al contrario se sono molto differenti tra di loro otterremo un risultato molto piccolo. Per cui, se per esempio la prima matrice 'appiattita' corrisponde all'intonazione e la seconda corrisponde al volume ed otteniamo un prodotto scalare elevato vorrà dire che quando il volume è alto anche l'intonazione è alta. Il prodotto scalare però può fornirci ovviamente numeri molto grandi. Allora li normalizziamo dividendo ogni elemento per il numero totale di elementi nella matrice

In [5]:
class GramMatrix(nn.Module):

  def forward(self, input):
    # a = batch size (= 1)
    # b = numero di feature maps
    # (c, d) = dimensione di una feature map (N = c * d)
    a, b, c = input.size()
    features = input.view(a * b, c)
    # calcoliamo la Gram Matrix
    G = torch.mm(features, features.t())
    # normalizziamo i valori della Gram Matrix
    # dividendo per il numero di elementi in ogni feature map
    return G.div(a * b * c)
  
class StyleLoss(nn.Module):

  def __init__(self, target, weight):
    super(StyleLoss, self).__init__()
    self.target = target.detach() * weight
    self.weight = weight
    self.gram = GramMatrix()
    self.criterion = nn.MSELoss()

  def forward(self, input):
    self.output = input.clone()
    self.G = self.gram(input)
    self.G.mul_(self.weight)
    self.loss = self.criterion(self.G, self.target)
    return self.output

  def backward(self, retain_graph=True):
    self.loss.backward(retain_graph=retain_graph)
    return self.loss

# Conversione da Wav a Matrice

Utilizzeremo `librosa` per convertire i nostri file `.wav` in matrici da poter passare a PyTorch. Eseguiamo una Short-Time Fourier Transform ([STFT](https://en.wikipedia.org/wiki/Short-time_Fourier_transform))

In [18]:
N_FTT = 2048 # window size

def read_audio_spectrum(filename):
  x, fs = librosa.load(filename)
  S = librosa.stft(x, N_FTT)
  p = np.angle(S)
  S = np.log1p(np.abs(S))
  return S, fs

style_audio, style_sr = read_audio_spectrum(style_audio_name)
content_audio, content_sr = read_audio_spectrum(content_audio_name)

if (content_sr != style_sr):
  raise 'Campionamento diverso tra le fonti audio'

style_audio = style_audio.reshape([1, 1025, style_audio.shape[1]])
content_audio = content_audio.reshape([1, 1025, content_audio.shape[1]])

if torch.cuda.is_available():
  style_float = Variable((torch.from_numpy(style_audio)).cuda())
  content_float = Variable((torch.from_numpy(content_audio)).cuda())
else:
  style_float = Variable((torch.from_numpy(style_audio)))
  content_float = Variable((torch.from_numpy(content_audio)))